### Retrieval Augmented Generation (RAG) PoC with Python sdk and SAP HANA Vector DB


#### Pre-requisite:

Use the secrets folder to store your service key credentials. Credentials required for:  
* Access to the SAP GenAI XL
* Access to the HanaDB

This guide does not illustrate how to generate embeddings using the AI Core proxy embedding model. This part is coverd in the other notebooks: 

* Generate-and-store-embeddings_with-HanaDB-AICore-RestAPI.ipynb
* Generate-and-store-embeddings_with-HanaDB-AICore-PythonSDK.ipynb

#### Introduction: 
In this guide we will be using a dataset that already includes the embeddings in the 'VECTOR_STR' column which has been generated for the 'TEXT' column using the text-embedding-ada-002 model. 
We will store these embeggings as REAL_VECTORS inside the SAP HANA DB and use the vector search functionality to build a Retrieval Augmented Generation (RAG) usecase with AI Core proxy LLMs. 

#### Step-by-step guide:
* Loading your data from csv 
* Connection with Hana database
* Create a new Hana table and push data into it
* Add a new column of data type REAL_VECTOR to your data table 
* Use the TO_REAL_VECTOR function to convert the embeggings to Real Vectors (VECTOR_RE). (This is necessary for the HANA DB to understand the embeddings.) and Update the data table.
* Connection with AI Core proxy LLMs through python sdk
* Levarage the similarity search functions which HANA DB offers for retreving relevant context based on a query.
* The context can then be used to formulate a prompt which is fed to an AI Core proxy Chat LLM.

In [ ]:
# !pip install pandas
# !pip install hana_ml
# !pip install "sap-llm-commons[all]" see https://github.tools.sap/AI-Playground-Projects/llm-commons
# !pip install langchain
# add config.json according to https://github.tools.sap/AI-Playground-Projects/llm-commons/tree/main/docs/proxy

#### 1. Loading your data

In [1]:
# import some vector data from csv
import pandas as pd
import hana_ml
df = pd.read_csv('./data/GRAPH_DOCU_QRC3.csv', low_memory=False)
df.head(3)

ModuleNotFoundError: No module named 'shapely'


,ID,L1,L2,L3,FILENAME,HEADER1,HEADER2,TEXT,VECTOR_STR
0,273,90,40,0,090-040-000-Appendix_C_-_GraphScript_Cheat_She...,Appendix C - GraphScript Cheat Sheet,Weighted Path Functions,<!--! subsection -->\n### WEIGHT \n```graphsc...,"[0.015699435,0.020284351,0.0003677337,-0.00413..."
1,52,60,20,30,060-020-030-Basic_Vertex_Operations.md,Basic Vertex Operations,DEGREE,Returns the number of incoming and outgoing ed...,"[0.018821003,0.012627394,-0.007940338,-0.00959..."
2,44,60,20,20,060-020-020-Basic_Graph_Operations.md,Basic Graph Operations,EDGES,Returns all edges in a graph. \n- EDGES(GRAPH...,"[-0.013607875,0.009249507,-0.03403819,-0.03394..."


#### 2. Connection to HANA Database

In [2]:
import json
with open('data/secrets/ies-hana-vectordb-schema-poc-sk.json', 'r') as f:
    hana_service_key = json.load(f)

In [3]:
from hana_ml import ConnectionContext

# cc = ConnectionContext(userkey='VDB_BETA', encrypt=True)
cc= ConnectionContext(
    address=hana_service_key['host'],
    port=hana_service_key['port'],
    user=hana_service_key['user'],
    password=hana_service_key['password'],
    currentSchema=hana_service_key['schema'],
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1710842063 (CE2024.10)
USR_5SKS2ZNTSKBBRAFPULSZIT6NR


#### 3. Create a new Hana table and push data into it

In [4]:
# Create a table
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE GRAPH_DOCU_QRC3(ID BIGINT, L1 NVARCHAR(3), L2 NVARCHAR(3), L3 NVARCHAR(3), FILENAME NVARCHAR(100), HEADER1 NVARCHAR(5000), HEADER2 NVARCHAR(5000), TEXT NCLOB, VECTOR_STR NCLOB);'''
cursor.execute(sql_command)
cursor.close()

In [5]:
# Upload data into the hana table
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df,
    table_name="GRAPH_DOCU_QRC3",
    allow_bigint=True,
    append=True
    )

100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


#### 4. Add a new column of data type REAL_VECTOR to your data table ((Let us call this column: VECTOR))

In [6]:
# Add REAL_VECTOR column
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE GRAPH_DOCU_QRC3 ADD (VECTOR REAL_VECTOR(1536));'''
cursor.execute(sql_command)
cursor.close()

#### 5. Use the TO_REAL_VECTOR function to convert the embeggings to Real Vectors and Update the data table.

In [7]:
# Create vectors from embedding strings
cursor = cc.connection.cursor()
sql_command = '''UPDATE GRAPH_DOCU_QRC3 SET VECTOR = TO_REAL_VECTOR(VECTOR_STR);'''
cursor.execute(sql_command)
cursor.close()

#### 6. Connection with AI Core proxy LLMs through llm-commons python-sdk

In [8]:
# Read ML deployed model
with open('data/secrets/genai-xl-test-instance.json') as f:
    sk = json.load(f)

In [9]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

# specify proxy version
llm_commons.proxy.base.proxy_version = 'aicore'

In [10]:
resource_group = '3f7513e0-0c3e-4dbf-a523-04f78fa295ca'

os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [11]:
from llm_commons.proxy.identity import AICoreProxyClient

aic_proxy_client = AICoreProxyClient()
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc008d860d221c90', config_id='bcaa04ee-bb2e-4e7f-b44f-4c374d2a42eb', config_name='gpt-4-ptu-config', deployment_id='dc008d860d221c90', model_name='gpt-4', created_at=datetime.datetime(2024, 3, 26, 9, 19, 5), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '0613PTU'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/deac9533e2d3dc51', config_id='b180ebf2-9cb1-4e86-9cc5-6f5929d0c35b', config_name='gpt-35-turbo-config', deployment_id='deac9533e2d3dc51', model_name='gpt-35-turbo', created_at=datetime.datetime(2024, 3, 26, 9, 15, 47), additonal_parameters={'executable_id': 'azure-openai', 'model_version': 'latest'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/da6a26f83dc8e241', config_id='b6edfd37-8f3

In [12]:
# Initialize the Embedding model: Replace the deployment_id with your resource group deployment id for 'text-embedding-ada-002-v2'
from llm_commons.langchain.proxy import init_embedding_model, init_llm
embedding_model = init_embedding_model('text-embedding-ada-002-v2', 
                                 proxy_client=aic_proxy_client, 
                                 deployment_id='da6a26f83dc8e241', 
                                 api_base=llm_commons.proxy.api_base)

In [14]:
# Initialize the Chat model: Replace the deployment_id with your resource group deployment id for 'gpt-35-turbo'

llm = init_llm('gpt-35-turbo', 
               proxy_client = aic_proxy_client,
               temperature=0., 
               max_tokens=256, 
               deployment_id='deac9533e2d3dc51', 
               api_base=llm_commons.proxy.api_base)

#### 7. Levarage the similarity search functions which HANA DB offers for retreving relevant context based on a query.

SAP HANA VectrDB provides two distance calculating similarity search functions: L2Distance() and cosine_similarity(), to enhance the platform's capability to compute vector similarity.

In [15]:
# Wrapping HANA vector search in a function: Here we are using the Cosine Similarity as the Similarity Search function. 
def run_vector_search(query: str, embedding_model, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = embedding_model.embed_query(query)
    sql = '''SELECT TOP {k} "ID", "HEADER1", "HEADER2", "TEXT"
        FROM "GRAPH_DOCU_QRC3"
        ORDER BY "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    # context = ' '.join(df_context['TEXT'].astype('string'))
    return df_context

In [16]:
# Test the vector search
query = "How can I run a shortest path algorithm?"
df_context = run_vector_search(query=query, embedding_model=embedding_model,metric="COSINE_SIMILARITY",k=5)
df_context

,ID,HEADER1,HEADER2,TEXT
0,211,Complex GraphScript Examples,GraphScript Procedure Example,The following example depicts a more complex e...
1,90,Graph Traversal Statements,Dijkstra's Algorithm (DIJKSTRA),DIJKSTRA searches for shortest paths in a weig...
2,83,Built-In Graph Algorithms,Shortest Path,```bnf\n<sssp_function> ::= SHORTEST_PATH '(' ...
3,65,Basic Weighted Path Operations,(Constructors),WEIGHTEDPATH objects can’t be constructed dire...
4,24,GraphScript Language,None,GraphScript is an imperative programming langu...


#### 8. Using langchain framework for prompt templates

In [17]:
# Prompt. Do also use your knowledge from outside the given context.
promptTemplate_fstring = """
You are an SAP HANA Cloud expert.
You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end.

Context:
{context}

Question:
{query}
"""

In [18]:
from langchain.prompts import PromptTemplate
promptTemplate = PromptTemplate.from_template(promptTemplate_fstring)

In [19]:
# The ask_llm function takes in the user query and converts them to embeddings first. Then a vector search is performed using the chosen metric and a context is retrieved.
# The context is then leveraged to create a prompt using the langchains PrompTemplate class. The propt is then fed as input to a chat completion LLM which provides relevant response.

def ask_llm(query: str, embedding_model, chat_model ,retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 6) -> str:

    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        print(color.RED + 'Running retrieval augmented generation.' + color.END)
        print(color.RED + '\nEmbedding the query string and running HANA vector search.' + color.END)
        context = run_vector_search(query, embedding_model, metric, k)
        print(context)
        print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
    else:
        print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)
    prompt = promptTemplate.format(query=query, context=' '.join(df_context['TEXT'].astype('string')))
    #prompt = promptTemplate.format(query=query, context=context)    
    print(color.RED + '\nAsking LLM...' + color.END)

    llm = chat_model
    
    #llm = ChatOpenAI(deployment_id="gpt-4", temperature=0)

    response = llm.predict(prompt)
    
    print(color.RED + '...completed.' + color.END)
    print(color.RED + '\nQuery: ' + color.END, query)
    print(color.BLUE + '\nResponse:' + color.BLUE)
    print(response)

In [20]:
# query = "Can you define a HANA graph workspace on a JSON document store collection?"
#query = "How can I define a HANA graph workspace on a JSON document store collection?"
#query = "How do you run a shortest path algorithm in SAP HANA Graph engine?"
# query = "How can I run community detection Louvain in SAP HANA Graph?"
# query = "How can I run a BFS traversal in HANA Cloud"
query = "I want to calculate a shortest path. How do I do that?"

response = ask_llm(query=query, 
                   embedding_model=embedding_model,
                   chat_model= llm, 
                   retrieval_augmented_generation=True)

Running retrieval augmented generation.

Embedding the query string and running HANA vector search.
    ID                         HEADER1                          HEADER2  \
0   90      Graph Traversal Statements  Dijkstra's Algorithm (DIJKSTRA)   
1   83       Built-In Graph Algorithms                    Shortest Path   
2  211    Complex GraphScript Examples    GraphScript Procedure Example   
3   65  Basic Weighted Path Operations                   (Constructors)   
4   84       Built-In Graph Algorithms        Shortest Paths One-to-All   
5   85       Built-In Graph Algorithms                 K-shortest Paths   

                                                TEXT  
0  DIJKSTRA searches for shortest paths in a weig...  
1  ```bnf\n<sssp_function> ::= SHORTEST_PATH '(' ...  
2  The following example depicts a more complex e...  
3  WEIGHTEDPATH objects can’t be constructed dire...  
4  ```\n<spoa_function> ::= SHORTEST_PATHS_ONE_TO...  
5  ```bnf\n<ksp_function> ::= K_SHORTEST_PAT